# 빅분기 작업형2 Baseline
- 결과창이 시각적으로 보기 좋지 않으니, print()에 항상 `, "\n"` 넣어주기

In [ ]:
import pandas as pd
import numpy as np

# 0. 데이터 로드
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

print(train.shape)
print(train.head(), "\n")

print(test.shape)
print(test.head(), "\n")



# 1. EDA
# 1-1) train EDA
print(train.info(), "\n")
print(train.isnull().sum(), "\n")
print(train.describe(), "\n")

# 1-2) test EDA
print(test.info(), "\n")
print(test.isnull().sum(), "\n")
print(test.describe(), "\n")

# 범주형 칼럼 고유값 비교
print(train.descibe(include="O"), "\n")
print(test.describe("include="O), "\n")



# 2. 전처리
# 2-1) 결측치 처리, 불필요 칼럼 제거, X와 y 분리

# 2-1-1)중앙값으로 결측치 대체
train["결측 존재 칼럼"] = train["결측 존재 칼럼"].filna(train["결측 존재 칼럼"].median()) 
test["결측 존재 칼럼"] = test["결측 존재 칼럼"].fillna(test["결측 존재 칼럼"].median())

# 결측치 채워졌나 확인
print(train.isnull().sum(), "\n")
print(test.isnull().sum(), "\n")

# 2-1-2) id 같은 불필요 칼럼 제거
train = train.drop("id", axis=1)
print(train.head(1), "\n")

test_id = test.pop("id") # 결과 제출할 때 test_id는 필요할 수 있음
print(test.head(1), "\n")

# 2-1-3) X와 y분리
y = train.pop("예측할 타겟 칼럼")
print(y.head(1), "\n")



# 2-2) 스케일링 
from sklearn.preprocesiing import StandardScaler
scaler = StandardScaler()

# 연속형 칼럼 선택
print(train.info(), "\n")
con_cols = train.select_dtypes(exclude="object").copy().columns

train[con_cols] = scaler.fit_transform(train[con_cols])
test[con_cols] = scaler.transform(test[con_cols])

print(train.head(), "\n") # 데이터창이랑 결과창 비교해보기
print(test.head(), "\n")



# 2-3) 인코딩 
train = pd.get_dummies(train)
test = pd.get_dummies(test)

test = test[train.columns] # 칼럼 순서 맞춰주기

print(train.head(1), "\n")
print(test.head(1), "\n")


# 3. 검증 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, 
                                                  y, 
                                                  random_state=1, 
                                                  stratify=y) # 분류일 때만 !! 

print(X_train.shape, X_val.shape)



# 4. 모델링 : 랜덤포레스트, lgbm 2개 준비
# 4-1) 성능지표
# 분류 성능지표
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

# 회귀 성능지표
from sklearn.metrics import mean_square_error, r2_score



# 4-2) 분류 모델
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=1)
rf.fit(X_train, y_train)

rf_pred = rf.predict(X_val) 
rf_pred_proba = rf.predict_proba(X_val)[:, 1]

# roc_auc_score
rf_roc = roc_auc_score(y_val, rf_pred_proba)

# f1_score
rf_f1 = f1_score(y_val, rf_pred, 
                 average = "macro") # 다중분류일 때

# accuracy_score
rf_acu = accuracy_score(y_val, rf_pred)



# lgbm
from lightgbm import LGBMClassifier
from sklearn.ensemble import LGBMClassifier
lgbm = LGBMClassifier(random_state=1)
lgbm.fit(X_train, y_train)

lgbm_pred = lgbm.predict(X_val) # 클래스에 대한 예측값
lgbm_pred_proba = lgbm.predict_proba(X_val)[:, 1] # 클래스 1에 대한 예측 '확률'

# roc_auc_score
lgbm_roc = roc_auc_score(y_val, lgbm_pred_proba)

# f1_score
lgbm_f1 = f1_score(y_val, lgbm_pred,
                   average = "macro") # 다중분류일 때

# accuracy_score
lgbm_acu = accuracy_score(y_val, lgbm_pred)



# 4-3) 회귀 모델
# 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=1)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)

# rmse
rf_rmse = np.sqrt(mean_square_error(y_val, rf_pred))

# r2_score
rf_r2 = r2_score(y_val, rf_pred)



# lgbm
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor(random_state=1)
lgbm.fit(X_train, y_train)
lgbm_pred = lgbm.predict(X_val)

# rmse
lgbm_rmse = np.sqrt(mean_square_error(y_val, lgbm_pred))

# r2_score
lgbm_r2 = r2_score(y_val, lgbm_pred)



# 최종 모델 선택하여 pred 지정
# 분류일 때 예시
# 클래스 예측
pred = lgbm.predict(X_val)

# 클래스 예측 확률
pred = lgbm.predict_proba(X_val)[:, 1] # 정확히 클래스가 어떤 경우의 확률을 말하는지 문제 꼼꼼히 읽을 것 

# 회귀일 때 예시
pred = lgbm.predict(X_val)



# 5. 제출 : df, csv
submit = pd.DataFrame({"id" : test_id, "pred / 제출 칼럼명 자세히 보기" : pred})
submit.to_csv("수험번호.csv", index=False)

check = pd.read_csv("수험번호.csv") # 제출 잘 됐는지 확인